In [8]:
import time
import os

import cv2
# codespace에서 opencv 문제 발생 시 해결 방법
# https://somjang.tistory.com/entry/Docker-opencv-%EC%84%A4%EC%B9%98-%EC%8B%9C-ImportError-libGLso1-cannot-open-shared-object-file-No-such-file-or-directory-%ED%95%B4%EA%B2%B0-%EB%B0%A9%EB%B2%95

import ray
import numpy as np

### single-core 동작 시 시간

In [15]:
num_cpus = 3

filters = [np.random.normal(size=(4, 4)) for _ in range(num_cpus)]
aver_ptime = 0

for _ in range(10):
    start = time.time()
    image = np.zeros((3000, 3000))
    result = []
    
    for i in range(num_cpus):
        result.append(cv2.filter2D(image, -1, filters[i]))
        
    ptime = time.time() - start
    aver_ptime += ptime / 10
    
    print(f"{ptime:.3f}s")
    
print(f"average ptime: {aver_ptime:.3f}s")

0.335s
0.283s
0.212s
0.234s
0.206s
0.205s
0.231s
0.211s
0.210s
0.210s
average ptime: 0.234s


### 3-core로 동작 시 걸리는 시간

* codespace 환경에서 core 수를 고려하지 않고 3-core로 동작시켰을 때 성능이 저하되는 이슈가 발생하였습니다.

In [14]:
num_cpus = 3
ray.init(num_cpus=num_cpus, ignore_reinit_error=True)

@ray.remote
def f(image, kernel):
    return cv2.filter2D(image, -1, kernel)

filters = [np.random.normal(size=(4, 4)) for _ in range(num_cpus)]

aver_ptime = 0

for _ in range(10):
    start = time.time()
    image = np.zeros((3000, 3000))
    image_id = ray.put(image)
    
    result = ray.get([f.remote(image_id, filters[i]) for i in range(num_cpus)])
    
    ptime = time.time() - start
    aver_ptime += ptime / 10
    
    print(f"{ptime:.3f}s")

print(f"aver_ptime: {aver_ptime:.3f}s")

1.805s
2.005s
0.405s
2.257s
0.411s
0.755s
2.808s
1.332s
1.793s
1.371s
aver_ptime: 1.494s


### 6-Core

* 6-core로 codespace에서 동작시키니까 worker들이 주기적으로 죽어버리는 현상을 발견하였습니다.

In [18]:
num_cpus = 6
ray.init(num_cpus=num_cpus, ignore_reinit_error=True)

@ray.remote
def f(image, kernel):
    return cv2.filter2D(image, -1, kernel)

filters = [np.random.normal(size=(4, 4)) for _ in range(num_cpus)]

aver_ptime = 0

for _ in range(10):
    start = time.time()
    image = np.zeros((3000, 3000))
    image_id = ray.put(image)
    
    result = ray.get([f.remote(image_id, filters[i]) for i in range(num_cpus)])
    
    ptime = time.time() - start
    aver_ptime += ptime / 10
    
    print(f"{ptime:.3f}s")

print(f"aver_ptime: {aver_ptime:.3f}s")

19.981s


2023-05-04 06:03:31,509	WARNING worker.py:1986 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: 77f475bdb4638702e2952d400c3b99ca09c2afaa01000000 Worker ID: 5c9bee404acd6936a210e5693aa0172d00715f46c1f1e33b5ef013eb Node ID: 8adf337ff7866bdf5bdd5bda4bf94cd296b18083c225f209c0c9ccb8 Worker IP address: 172.16.5.4 Worker port: 45849 Worker PID: 26596 Worker exit type: SYSTEM_ERROR Worker exit detail: The leased worker has unrecoverable failure. Worker is requested to be destroyed when it is returned.
2023-05-04 06:03:31,529	WARNING worker.py:1986 -- WARNING: 8 PYTHON worker processes have been started on node: 8adf337ff7866bdf5bdd5bda4bf94cd296b18083c225f209c0c9ccb8 with address: 172.16.5.4. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
202

24.865s


2023-05-04 06:03:58,416	WARNING worker.py:1986 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: d536273453f723b84d54fec405b6c7bf3d753df101000000 Worker ID: 06522b03d804b944053de86f1d3a6213507291eaf8d455a82279c82d Node ID: 8adf337ff7866bdf5bdd5bda4bf94cd296b18083c225f209c0c9ccb8 Worker IP address: 172.16.5.4 Worker port: 38331 Worker PID: 29611 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker unexpectedly exits with a connection error code 2. End of file. There are some potential root causes. (1) The process is killed by SIGKILL by OOM killer due to high memory usage. (2) ray stop --force is called. (3) The worker is crashed unexpectedly due to SIGSEGV or other unexpected errors.


30.027s


KeyboardInterrupt: 

2023-05-04 06:04:29,844	WARNING worker.py:1986 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: 411c0ff2d0ad9535c3648cf097b08bc1e145363601000000 Worker ID: cbe261dcb6805875bb651c52d6a84fdafee7befe05171b8292cb423b Node ID: 8adf337ff7866bdf5bdd5bda4bf94cd296b18083c225f209c0c9ccb8 Worker IP address: 172.16.5.4 Worker port: 40215 Worker PID: 29819 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker unexpectedly exits with a connection error code 2. End of file. There are some potential root causes. (1) The process is killed by SIGKILL by OOM killer due to high memory usage. (2) ray stop --force is called. (3) The worker is crashed unexpectedly due to SIGSEGV or other unexpected errors.


### Result

* 병렬처리 환경에서 CPU 코어수 및 싱글코어 당 런타임 시 점유하는 CPU 사용률을 고려하여 Ray의 CPU 개수를 조정해야 합니다.